# Data Import

In [1]:
#Import required libraries


import numpy as np
import pandas as pd
import geopandas as gpd

#In order to draw in jupyter you need to use this command
import matplotlib.pyplot as plt
import seaborn as sns
corlor = sns.color_palette()
sns.set_style('darkgrid') #Set the drawing background



from scipy import stats
from scipy.stats import norm, skew 
#Statistical: normal distribution, skewness


#Limited to three decimal places
pd.set_option('display.float_format', lambda x:'{:.3f}'.format(x))

from subprocess import check_output

In [2]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
#Linear regression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.metrics import mean_squared_error, make_scorer
import statsmodels.api as sm

In [4]:
#XGBoost need
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_log_error, r2_score

In [5]:
#Import and put the data into pandas
train_df = pd.read_csv('/kaggle/input/train-df/training_dataset-1.csv')
test_df = pd.read_csv('/kaggle/input/test-df/test_dataset-1.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/train-df/training_dataset-1.csv'

# Data Structure & A summary statistic of the data 

In [ ]:
#Check the data structure, check if there are missing values, and check the data type
print("\nTraining Data Info:")
train_df.info()

In [ ]:
#View the first 5 rows of data in the training set
print("Traning data:")
train_df.head(5)

In [ ]:
#Classify features according to data type to facilitate subsequent processing
train_df.groupby(train_df.dtypes,axis=1).apply(lambda x:x.columns)

·id - Unique ID for each home sold 
·date - Date of the home sale 
·price - Price of each home sold 
·bedrooms - Number of bedrooms 
·bathrooms - Number of bathrooms, where .5 accounts for a room with a toilet but no shower 
·sqft_living - Square footage of the apartments interior living space 
·sqft_lot - Square footage of the land space 
·floors - Number of floors 
·waterfront - A dummy variable for whether the apartment was overlooking the waterfront or not 
·view - An index from 0 to 4 of how good the view of the property was 
·condition - An index from 1 to 5 on the condition of the apartment, 
·grade - An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design. 
·sqft_above - The square footage of the interior housing space that is above ground level 
·sqft_basement - The square footage of the interior housing space that is below ground level 
·yr_built - The year the house was initially built 
·yr_renovated - The year of the house’s last renovation 
·zipcode - What zipcode area the house is in 
·lat - Lattitude 
·long - Longitude 
·sqft_living15 - The square footage of interior housing living space for the nearest 15 neighbors 
·sqft_lot15 - The square footage of the land lots of the nearest 15 neighbors 

In [ ]:
#Check the data size before deleting the id
print("\nThe train data size before dropping id feature is: {}".format(train_df.shape))
print("\nThe test data size before dropping id feature is: {}".format(test_df.shape))

In [ ]:
# Selecting Numeric Features
numeric_features = train_df.select_dtypes(include=[np.number])

# Calculate descriptive statistics and transpose
numeric_stats = numeric_features.describe().T  
# Transpose the descriptive statistics and store them in numeric_stats

# Add median
numeric_stats['median'] = numeric_features.median()  

# Print statistics of numeric features
print("Statistics of numerical features:")
print(numeric_stats)

In [ ]:
# Histogram: Displays the distribution of housing prices
plt.subplot(1, 2, 1)
sns.histplot(train_df['price'], bins=30, kde=True)
plt.title('Distribution of House Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')

In [ ]:
# Set the style of the graph
sns.set(style="whitegrid")

# Create a box plot
plt.figure(figsize=(8, 6))
sns.boxplot(y=train_df['price'])  
# draw a box plot of price using the y-axis
plt.title('Boxplot of House Prices')
plt.ylabel('Price')
plt.xlabel('House Price')
plt.grid(axis='y')  
# Add y-axis grid lines
plt.show()

### 

 ### Create Price Heatmap using Location Information

In [ ]:
# Create Price Heatmap using Latitudes and Longitudes
# import pandas as pd
import folium
from folium.plugins import HeatMap

# Load your dataset
data = train_df

# Filter out rows where price or lat/long might be missing
data = data.dropna(subset=['lat', 'long', 'price'])

# Initialize a map centered around the average location
map_center = [data['lat'].mean(), data['long'].mean()]
m = folium.Map(location=map_center, zoom_start=10, tiles="OpenStreetMap")

# Prepare data for the heatmap: [latitude, longitude, weight (price)]
heat_data = [[row['lat'], row['long'], row['price']] for index, row in data.iterrows()]

# Create a heatmap layer without max_val
HeatMap(heat_data, min_opacity=0.2, max_zoom=15, radius=10, blur=15).add_to(m)

# Display the map
m.save("price_heatmap.html")

In [ ]:
# Generate a heatmap of zipcode & housing prices per square meter

# Create a new DataFrame to avoid affecting the original data
temp_df = train_df.copy()

# Calculate the price to living area ratio for each zipcode
temp_df['price_per_sqft'] = temp_df['price'] / temp_df['sqft_living']
avg_price_per_zipcode = temp_df.groupby('zipcode')['price_per_sqft'].mean().reset_index()

# Convert the zipcode column to string type to match gdf_zipcodes
avg_price_per_zipcode['zipcode'] = avg_price_per_zipcode['zipcode'].astype(str)

# Read the zipcode's geographic boundary data
zipcode_shapefile = '/kaggle/input/zipcode/Zipcodes.geojson' 
# Replace with everyone's own file path
gdf_zipcodes = gpd.read_file(zipcode_shapefile)

# Merge geographic data with average price to living area ratio data
gdf_merged = gdf_zipcodes.merge(avg_price_per_zipcode, left_on='ZIPCODE', right_on='zipcode', how='left')

# Set the color of the heat map
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
gdf_merged.boundary.plot(ax=ax, linewidth=1, color='black')  
# Draw the border
gdf_merged.plot(column='price_per_sqft', ax=ax, legend=True,
                legend_kwds={'label': "Average Price per Sqft by Zipcode",
                             'orientation': "horizontal"},
                cmap='coolwarm', missing_kwds={"color": "lightgrey"})  
# Missing values are set to gray

# Set the latitude and longitude range
ax.set_xlim(-122.50, -121.25)  
# Longitude range
ax.set_ylim(47.1, 47.8)        
# Latitude range

plt.title('Average Price per Square Foot by Zipcode')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()


# Data Preprocessing

In [ ]:
#Data processing on outliers and null values
#For numeric variables, use y to do a scatterplot of house prices, find outliers and delete abnormal data
def plot_data_scatterplot_for_train_df(x):
    
   plt.figure(figsize=(8, 6))
   sns.scatterplot(x=x, y='price', data=train_df)
   plt.show()

In [ ]:
train_df.columns[train_df.dtypes != 'object']

**Outliers**

In [ ]:
#columns don't have outliers
def plot_data_scatterplot_with_boxplot(column):
    # Create a two-column chart layout
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [3, 1]})
    
    # Draw a scatter plot on the left
    sns.scatterplot(data=train_df, x=column, y='price', ax=axes[0])
    axes[0].set_title(f'Scatter Plot of {column} vs price')
    
    # Draw a box plot on the right
    sns.boxplot(data=train_df, y=column, ax=axes[1])
    axes[1].set_title(f'Box Plot of {column}')
    
    plt.tight_layout()
    plt.show()

# Use the new function to draw scatter plots and box plots of columns
columns_to_plot = ['floors', 'long', 'waterfront', 'view', 'condition', 'grade', 
                   'yr_built', 'yr_renovated', 'zipcode', 'lat']
for column in columns_to_plot:
    plot_data_scatterplot_with_boxplot(column)


In [ ]:
# Columns have outliers
def plot_data_scatterplot_with_boxplot(column):
    # Create a two-column chart layout
    fig, axes = plt.subplots(1, 2, figsize=(12, 5), gridspec_kw={'width_ratios': [3, 1]})
    
    # Draw a scatter plot on the left
    sns.scatterplot(data=train_df, x=column, y='price', ax=axes[0])
    axes[0].set_title(f'Scatter Plot of {column} vs price')
    
    # Draw a box plot on the right
    sns.boxplot(data=train_df, y=column, ax=axes[1])
    axes[1].set_title(f'Box Plot of {column}')
    
    plt.tight_layout()
    plt.show()

# Use the new function to draw scatter plots and box plots of columns
columns_to_plot = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 
                   'sqft_living15', 'sqft_lot15']
for column in columns_to_plot:
    plot_data_scatterplot_with_boxplot(column)

In [ ]:
train_df.query('bedrooms> 15')

In [ ]:
train_df.query('bathrooms> 7')

In [ ]:
train_df.query('sqft_living> 12000')

In [ ]:
train_df.query('sqft_lot> 1500000')

In [ ]:
train_df.query('sqft_above> 8000')

In [ ]:
train_df.query('sqft_basement> 3000')

In [ ]:
train_df.query('sqft_living15 > 6000')

In [ ]:
train_df.query('sqft_lot15 > 500000')

In [ ]:
#Delete house records with outlier IDs
values = [2402100895, 1225069038, 6762700020, 9208900037, 424049043, 1020069017, 7767000060, 1924059029, 2524069078, 225079036, 3420069060 ]
train_df = train_df[train_df.id.isin(values) == False]

In [ ]:
print("After deletion: {}".format(train_df.shape))

**Missing Values**

In [ ]:
#Handling missing values
#Check whether the dependent variable price has missing values
print(train_df['price'].isnull().sum())

In [ ]:
#Concat test and train data
# Reset the index before merging
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

all_data = pd.concat([train_df, test_df])
all_data.tail(10)
print("After combine all data: {}".format(all_data.shape))

In [ ]:
#Directly check whether there are missing values ​​in the data set, but do not display the price column
missing_values = all_data.filter(regex='^(?!price$)').isnull().sum()
print(missing_values)

In [ ]:
# Handling missing values in yr_renovated
all_data['yr_renovated'] = np.where(all_data['yr_renovated'] == 0, all_data['yr_built'], all_data['yr_renovated'])

# Check whether the replacement is successful
print("Unique values ​​of the 'yr_renovated' column in the processed training set:")
print(all_data['yr_renovated'].unique())

In [ ]:
# Re-split all_data into train_df and test_df based on updated size of train_df
train_df = all_data.iloc[:len(train_df), :].copy()
test_df = all_data.iloc[len(train_df):, :].copy()

#check date shape
print("\nThe train data size is: {}".format(train_df.shape))
print("\nThe test data size is: {}".format(test_df.shape))

In [ ]:
#Handling missing values in bedrooms
# show datas with bedrooms is 0
bedrooms_zero_train = train_df[train_df['bedrooms'] == 0]
bedrooms_zero_test = test_df[test_df['bedrooms'] == 0]
print("Bedrooms = 0:")
print(bedrooms_zero_train)
print(bedrooms_zero_test)

In [ ]:
# Delete rows with bedrooms ==0
train_df = train_df[train_df['bedrooms'] != 0]
test_df = test_df[test_df['bedrooms'] != 0]

#Check the data size after data cleaning for bedrooms == 0
print("\nThe train data size after data cleaning for bedrooms is: {}".format(train_df.shape))
print("\nThe test data size after data cleaning for bedrooms is: {}".format(test_df.shape))

In [ ]:
# show datas with bathrooms is 0
bathrooms_zero_train = train_df[train_df['bathrooms'] == 0]
bathrooms_zero_test = test_df[test_df['bathrooms'] == 0]
print("Bathrooms = 0:")
print(bathrooms_zero_train)
print(bathrooms_zero_test)

In [ ]:
# Delete rows with bathrooms == 0
train_df = train_df[train_df['bathrooms'] != 0]
test_df = test_df[test_df['bathrooms'] != 0]

#Check the data size after data cleaning for bathrooms == 0
print("\nThe train data size after data cleaning for bathrooms is: {}".format(train_df.shape))
print("\nThe test data size after data cleaning for bathrooms is: {}".format(test_df.shape))

# Feature Engineering

Special feature processing: zipcode and date

**zipcode**

In [ ]:
#calculate the unique value
unique_zipcodes = train_df['zipcode'].nunique()
print(f"Unique zipcodes: {unique_zipcodes}") 

In [ ]:
# Calculate the average house price/sqft_living for each zipcode in the training data
train_df['price_per_sqft'] = train_df['price'] / train_df['sqft_living']
zipcode_mean = train_df.groupby('zipcode')['price_per_sqft'].mean().reset_index()

In [ ]:
# Rename columns
zipcode_mean.columns = ['zipcode', 'zipcode_encoded']

In [ ]:
# Replace the zipcode field in the training data with the encoded value
train_df = train_df.merge(zipcode_mean, on='zipcode', how='left')
train_df['zipcode'] = train_df['zipcode_encoded']
train_df.drop(columns=['price_per_sqft', 'zipcode_encoded'], inplace=True)

In [ ]:
# apply it to the test dataset
# Merge encoded values into test data
test_df = test_df.merge(zipcode_mean, on='zipcode', how='left')

In [ ]:
# Check the merge results
print("Merged test data:")
print(test_df)

In [ ]:
# Replace the zipcode field in the test data with the encoded value
# Make sure we check if zipcode_encoded exists
if 'zipcode_encoded' in test_df.columns:
    test_df['zipcode'] = test_df['zipcode_encoded']
else:
    print("No 'zipcode_encoded' column found in the test data")

In [ ]:
# Remove columns that are no longer needed
test_df.drop(columns=['zipcode_encoded'], inplace=True, errors='ignore')

In [ ]:
# Display the results 
print("Training data:")
print(train_df)
print("\nTest data:")
print(test_df)

**date**

In [ ]:
# Use .loc to ensure operation on original DataFrame
train_df.loc[:, 'is_train'] = 1  
# Training set labeling
test_df.loc[:, 'is_train'] = 0   
# Test set labeling

# Merge two datasets
all_data = pd.concat([train_df, test_df], ignore_index=True)

In [ ]:
all_data.tail(10)

In [ ]:
# Convert the 'date' column to datetime type
all_data['date'] = pd.to_datetime(all_data['date'])

In [ ]:
# View the converted data type
print("\nThe converted data type:")
print(all_data.dtypes)

# Verify that the conversion was successful
print("\nExample of converted date:")
print(all_data['date'].head())

In [ ]:
# Extract year and month
all_data['year_sold'] = all_data['date'].dt.year
all_data['month_sold'] = all_data['date'].dt.month

In [ ]:
# Delete the original date column
all_data = all_data.drop(['date'], axis=1)

In [ ]:
# Split the dataset back to its original state
train_df = all_data[all_data['is_train'] == 1].drop('is_train', axis=1)
test_df = all_data[all_data['is_train'] == 0].drop(['is_train', 'price'], axis=1)

In [ ]:
# View Results
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# Check the processed shape
print(f"Training set: {train_df.shape}, Test set: {test_df.shape}")

**Feature transformation (e.g., log transformation)**

In [ ]:
# Check the shape and type of the price column
print("Shape:", train_df.price.shape)  
# should be (n,)
print("Type:", type(train_df.price))  
# should be a pandas Series

# View the previous rows of data
print("Sample Data:\n", train_df.price.head())

# Confirm data type
print("Data Types:\n", train_df.price.apply(type).value_counts())


In [ ]:
#check the distrubition of target variable price, in order to observe its skewness characteristics
plt.hist(train_df.price, bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Target Variable')
plt.xlabel('price')
plt.ylabel('Frequency')
plt.show()


The distribution of the target variable, price, shows significant skewness, indicating that it is not normally distributed. This lack of normality can negatively affect the performance of models that assume a normal distribution for optimal predictions.

To address this, applying a logarithmic transformation to the price variable will help to reduce the skewness and approximate a more normal distribution.

This transformation will be particularly beneficial when computing metrics in regression models, as it will ensure that errors and predictions are measured in a more balanced manner, leading to better results and model accuracy.

In [ ]:
# Convert price to log price, use .loc to ensure the operation is on the original DataFrame
train_df.loc[:, 'log_price'] = np.log(train_df['price'])

# View the results of the first few rows
print(train_df[['price', 'log_price']].head(5))


In [ ]:
# Check the shape of log_price
print(train_df['log_price'].shape)


In [ ]:
plt.hist(train_df.log_price, bins=30, edgecolor='black', alpha=0.7)
plt.title('Distribution of Target Variable log_price')
plt.xlabel('log_price')
plt.ylabel('Frequency')
plt.show()


In [ ]:
#The ID field is not used during processing, so delete it, but keep the ID for final output
print("The train data size before dropping ID feature is: {}".format(train_df.shape))
print("The test data size before dropping ID feature is: {}".format(test_df.shape))

#Keep the ID column
train_ID = train_df['id']
test_ID = test_df['id']

#Because the id is not necessary for prediction, delete it
train_df.drop("id", axis = 1, inplace = True)
test_df.drop("id", axis = 1, inplace = True)

#View the data size after deleting the id
print("\nThe train data size after dropping id feature is: {}".format(train_df.shape))
print("\nThe test data size after dropping id feature is: {}".format(test_df.shape))

In [ ]:
# Assume train is your dataset and already contains the 'log_price' column
train_df_corr = train_df.select_dtypes(include=[np.number])

# Calculate the correlation matrix
corr = train_df_corr.corr()

# Sort: first based on correlation with 'SalePrice' (or 'log_price')
corr_target_sorted = corr['log_price'].abs().sort_values(ascending=False).index

# Rearrange the correlation matrix according to the sorting results (also remove the 'price' column and row)
corr_sorted = corr.loc[corr_target_sorted, corr_target_sorted].drop('price', axis=1).drop('price', axis=0)

# Visualize heatmap and adjust annotation font size
plt.figure(figsize=(15, 10))
sns.heatmap(corr_sorted, annot=True, cmap='coolwarm', square=True, annot_kws={"size": 8})
plt.title('Correlation Matrix (sorted by correlation with log_price)')
plt.show()


grade, sqft_living, zipcode have the highest correlation with price:


The other 3 variables with a correlation higher than 0.5 with price are: sqft_living15, sqft_above, bathrooms.

sqft_living and sqft_above have a correlation of 0.88, meaning they move together strongly. It is multicollinearity. Why Multicollinearity is an Issue? When multicollinearity is present:

Redundancy: Both variables are conveying essentially the same information. Including both doesn't add value and might lead to confusing results.
Unstable Coefficients: The regression model may struggle to determine which variable (sqft_living or sqft_above) is more important, leading to unstable coefficient estimates. This means small changes in the data could cause large swings in the coefficients, making the model less reliable.

In [ ]:
# Import necessary libraries
sns.set()

# Define columns for the scatterplot
cols = ['price', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'zipcode', 'bathrooms', 
        'view', 'bedrooms', 'sqft_basement', 'lat', 'floors', 'waterfront', 'yr_renovated', 
        'sqft_lot', 'sqft_lot15', 'yr_built', 'condition', 'long', 'month_sold', 'year_sold']

# Create scatterplots only between 'price' and the other variables
sns.pairplot(train_df, x_vars=cols[1:], y_vars='price', height=2.5)
plt.show()

In [ ]:
# Import necessary libraries
sns.set()

# Define columns for the scatterplot
cols = ['log_price', 'sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'zipcode', 'bathrooms', 
        'view', 'bedrooms', 'sqft_basement', 'lat', 'floors', 'waterfront', 'yr_renovated', 
        'sqft_lot', 'sqft_lot15', 'yr_built', 'condition', 'long', 'month_sold', 'year_sold']

# Create scatterplots only between 'price' and the other variables
sns.pairplot(train_df, x_vars=cols[1:], y_vars='log_price', height=2.5)
plt.show()

In [ ]:
#scatterplot
sns.set()
cols = ['price', 'grade', 'zipcode', 'sqft_living', 'sqft_living15', 'sqft_above', 'bathrooms']
sns.pairplot(train_df[cols], size = 2.5)
plt.show();

This pair plot helps us explore the pairwise relationships between these variables, allowing us to visually inspect potential correlations, outliers, or patterns that could be useful for our house price prediction model

# Feature Selection and Model Selection 

**M1:use all features for linear regression**

In [ ]:
# Assume train_df is the training dataset
X = train_df.drop(columns=['log_price','price'])
y = train_df['log_price']

# Creating a Linear Regression Model
model = LinearRegression()

# Define RMSE scoring function
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Cross-validation using KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42) 
# 5-fold Cross-validation

# Cross-validation
scores = cross_val_score(model, X, y, scoring=rmse_scorer, cv=kf)

# Output cross-validation RMSE
print("Cross-validated RMSE scores:", scores)
print("Mean RMSE:", np.mean(scores))

# Fitting the final model
model.fit(X, y)

# Use statsmodels to refit the model and output the OLS regression results table
X_with_const = sm.add_constant(X)  
# Add a constant term (intercept term)
ols_model = sm.OLS(y, X_with_const).fit()  
# Fit the OLS model

# Output OLS regression results
print(ols_model.summary())

**M2: Based on VIF, eliminate multicollinearity between some independent variables and use some features for linear regression**

In [ ]:
# Assume train_df is your DataFrame, containing all features
# Selecting Numeric Features
train_df_corr = train_df.select_dtypes(include=[np.number])

# Calculate the VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = train_df_corr.columns
vif_data["VIF"] = [variance_inflation_factor(train_df_corr.values, i) for i in range(train_df_corr.shape[1])]

# Output VIF data
print(vif_data)

# Filter features whose VIF values exceed the threshold
threshold = 10  
high_vif_features = vif_data[vif_data["VIF"] > threshold]

# Display high VIF features
print("Features with VIF greater than", threshold)
print(high_vif_features)


In [ ]:
train_df.columns

In [ ]:
# Assume train_df is your training dataset
X = train_df.drop(columns=['log_price', 'price','sqft_above', 'sqft_basement', 'sqft_living15', 'bathrooms'])
y = train_df['log_price']

# Creating a Linear Regression Model
model = LinearRegression()

# Define RMSE scoring function
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Cross-validation using KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  
# 5-fold Cross-validation

# Cross-validation
scores = cross_val_score(model, X, y, scoring=rmse_scorer, cv=kf)

# RMSE
print("Cross-validated RMSE scores:", scores)
print("Mean RMSE:", np.mean(scores))

# Fitting the final model
model.fit(X, y)

# Use statsmodels to refit the model and output the OLS regression results table
# Here you need to add a constant term (intercept term) to X
X_with_const = sm.add_constant(X)

# Fitting an OLS model using statsmodels
ols_model = sm.OLS(y, X_with_const).fit()

# Output OLS regression results
print(ols_model.summary())

**M3: Remove insignificant independent variables after dealing with multicollinearity**

In [ ]:
# Assume train_df is the training dataset
X = train_df.drop(columns=['log_price', 'price','sqft_above', 'sqft_basement', 'sqft_living15', 'bathrooms','floors','sqft_lot15'])
y = train_df['log_price']

# Creating a Linear Regression Model
model = LinearRegression()

# Define RMSE scoring function
rmse_scorer = make_scorer(mean_squared_error, squared=False)

# Cross-validation using KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  
# 5-fold Cross-validatio

# Cross-validatio
scores = cross_val_score(model, X, y, scoring=rmse_scorer, cv=kf)

# print RMSE
print("Cross-validated RMSE scores:", scores)
print("Mean RMSE:", np.mean(scores))

# Fitting the final model
model.fit(X, y)

# Use statsmodels to refit the model and output the OLS regression results table
# Here you need to add a constant term (intercept term) to X
X_with_const = sm.add_constant(X)

# Fitting an OLS model using statsmodels
ols_model = sm.OLS(y, X_with_const).fit()

# Output OLS regression results
print(ols_model.summary())

In [ ]:
# Using the model to make predictions
y_pred_log_price = model.predict(X)

# Convert predicted log_price to price
y_pred_price = np.exp(y_pred_log_price)

# Convert actual log_price to price
y_actual_price = np.exp(y)

# Calculate RMSE on the price scale
rmse_price = mean_squared_error(y_actual_price, y_pred_price, squared=False)  
# squared=False return RMSE
print(f"Price Scale RMSE: {rmse_price:.5f}")


**M4 & M5: Try Lasso、Ridge**

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Separate features and target variable
X_train = train_df.drop(columns=['log_price', 'price'])
y_train = train_df['log_price']

X_test = test_df

# Identify categorical features for encoding
categorical_features = []
numeric_features = X_train.columns.difference(categorical_features)

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define models to evaluate
models = {
    'Lasso': LassoCV(alphas=np.logspace(-3, 2, 100), cv=5, random_state=42),
    'Ridge': RidgeCV(alphas=np.logspace(-3, 2, 100), cv=5),
}

for name, model in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
    # Cross-validation
    scores = cross_validate(pipeline, X_train, y_train, cv=5,
                            scoring=('neg_root_mean_squared_error', 'neg_mean_absolute_error', 'r2'),
                            return_train_score=True)
    
    print(f"\nModel: {name}")
    print(f"  RMSE: {-np.mean(scores['test_neg_root_mean_squared_error'])}")
    print(f"  MAE: {-np.mean(scores['test_neg_mean_absolute_error'])}")
    print(f"  R²: {np.mean(scores['test_r2'])}")
    # Fit the model on the entire training data to extract feature importance
    pipeline.fit(X_train, y_train)

    if name in ['Lasso', 'Ridge']:
        # Get feature names after preprocessing
        feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()
        # Get coefficients from the model
        coefficients = pipeline.named_steps['model'].coef_
        
        # For Lasso, filter out zero coefficients
        if name == 'Lasso':
            selected_features = feature_names[coefficients != 0]
            selected_coefficients = coefficients[coefficients != 0]
        else:
            selected_features = feature_names
            selected_coefficients = coefficients
        
        # Print all features and their coefficients for Lasso/Ridge
        for feature, coef in zip(selected_features, selected_coefficients):
            print(f"Feature: {feature}, Coefficient: {coef}")
        
        # Sort features by absolute coefficient value
        sorted_indices = np.argsort(np.abs(selected_coefficients))[::-1]
        sorted_features = selected_features[sorted_indices]
        sorted_coefficients = selected_coefficients[sorted_indices]

        # Plotting sorted feature importances for Lasso/Ridge
        plt.figure(figsize=(10, 6))
        plt.barh(sorted_features[:10], sorted_coefficients[:10])  # Plot top 10 features by absolute value of coefficient
        plt.xlabel('Coefficient Value')
        plt.title(f'Top Features by Coefficient - {name}')
        plt.gca().invert_yaxis()
        plt.show()

In [ ]:
print(f"Lasso  RMSE（Original）: {np.exp(0.18660178373477715) - 1}")

In [ ]:
print(f"Ridge  RMSE（Original）: {np.exp(0.1865838851032337) - 1}")

**M5: Random Forests**

Steps to Determine Feature Contributions

1. Train the Random Forest Model: Fit a Random Forest model to your training data.
2. Extract Feature Importances: Use the featureimportances attribute to get the importance score for each feature.
3. Sort and Display Top Features: Sort these scores to find the most important features and display them.

In [ ]:
# Random Forest feature importances
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', rf_model)])
pipeline_rf.fit(X_train, y_train)

print(f"Model: Random Forest")
print(f"  RMSE: {-np.mean(scores['test_neg_root_mean_squared_error'])}")
print(f"  R²: {np.mean(scores['test_r2'])}")
    
rf_feature_importance = rf_model.feature_importances_
rf_selected_features = feature_names[np.argsort(rf_feature_importance)[-10:]]  # Top 10 important features
print("Top Random Forest features:", rf_selected_features)

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': rf_feature_importance
})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Display top 10 features
print("Top 10 Features by Importance:")
print(feature_importance_df.head(10))

# Optionally, plot feature importances
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Feature'].head(10), feature_importance_df['Importance'].head(10))
plt.gca().invert_yaxis()
plt.xlabel('Importance')
plt.title('Top 10 Feature Importances in Random Forest')
plt.show()

**M6: XGBoost**

In [ ]:
# Prepare the data
X_xgb = train_df.drop(columns=["log_price","price"])  
# All features
y_xgb = train_df["log_price"]  
# Target variable

In [ ]:
#Split the dataset into training and testing sets
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X_xgb, y_xgb, test_size=0.2, random_state=42)

In [ ]:
# create XGBoost model
xgb_model = XGBRegressor()

In [ ]:
# Defining the hyperparameter grid
param_grid = {
    'max_depth': [4, 5, 6],  # Reduce the depth of the tree
    'learning_rate': [0.01, 0.05],  # Lower the learning rate
    'n_estimators': [100, 200, 300],  # Increase n_estimators
    'subsample': [0.7, 0.8],  # Using Subsets
    'reg_alpha': [0, 0.1, 0.5],  # L1 Regularization
    'reg_lambda': [1.0, 1.5]  # L2 Regularization
}

In [ ]:
# Hyperparameter tuning using random search
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_grid,
    n_iter=20,
    scoring='neg_mean_squared_log_error',
    cv=3,
    random_state=42
)
random_search.fit(X_train_xgb, y_train_xgb)


In [ ]:
# Get the best model
best_model = random_search.best_estimator_

In [ ]:
# Early Stopping
evals = [(X_train_xgb, y_train_xgb), (X_test_xgb, y_test_xgb)]  
# Monitoring training and test sets
best_model.fit(X_train_xgb, y_train_xgb, eval_set=evals, early_stopping_rounds=10, verbose=False)

In [ ]:
# Use the best model to make predictions on the test set
y_pred_xgb = best_model.predict(X_test_xgb)

In [ ]:
# calculate RMSE
rmse = mean_squared_error(y_test_xgb, y_pred_xgb, squared=False)  
# squared=False return RMSE
print(f"Score: {rmse:.5f} RMSE")

In [ ]:
# calculate R^2
r_squared = r2_score(y_test_xgb, y_pred_xgb)
print(f"R^2 Score: {r_squared:.5f}")

In [ ]:
# Print the best hyperparameters
print("Best parameters found: ", random_search.best_params_)

In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Draw a feature importance graph without displaying the default values
ax = xgb.plot_importance(best_model, importance_type='gain', show_values=False)
plt.title("Feature Importance (Gain)")

# Add text labels to the bar chart, formatted to two decimal places
for p in ax.patches:
    # Set the label text format to two decimal places
    label = f"{p.get_width():.2f}"
    # Set the label position, slightly offset to the right to avoid overlapping
    ax.annotate(label, 
                (p.get_width() + 0.01, p.get_y() + p.get_height() / 2), 
                ha='left', va='center', fontsize=9)

plt.show()

# Get feature importance and sort by importance
feature_importance = best_model.get_booster().get_score(importance_type='gain')
sorted_importance = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)
print("Feature importance (sorted by gain):")
for feature, importance in sorted_importance:
    print(f"{feature}: {importance:.4f}")

In [ ]:
# Convert predicted log_price to price
y_pred_price = np.exp(y_pred_xgb)

# Convert actual log_price to price
y_test_price = np.exp(y_test_xgb)

# Calculate RMSE on the price scale
rmse_price = mean_squared_error(y_test_price, y_pred_price, squared=False)  
# squared=False return RMSE
print(f"Price Scale RMSE: {rmse_price:.5f}")


# Best Model and Best Features

In [ ]:
# Create a dictionary with data
data = {
    'Model': [
        'Simple Linear Regression',
        'Linear Regression with Multicollinearity Handling',
        'Linear Regression with Multicollinearity Handling and Removal of Insignificant Variables',
        'Lasso',
        'Ridge',
        'Random Forest',
        'XGBoost'
    ],
    'RMSE': [0.1863, 0.1920, 0.1919, 0.1865, 0.1865, 0.1865, 0.1583],
    'R^2': [0.8740, 0.8670, 0.8670, 0.8737, 0.8738, 0.8738, 0.9103]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Set display options to format float numbers
pd.options.display.float_format = '{:.4f}'.format  # Use 4 decimal places

# Display the DataFrame
print(df)


We have chosen to use the **XGBoost model** for predicting house sale prices due to several important reasons:
1. Lowest RMSE: XGBoost has the lowest RMSE (0.1583), indicating the smallest prediction error among all models.
2. Highest R²: XGBoost has the highest R² value (0.9103), suggesting that it explains a significant portion of the variance in house sale prices.
3. Robustness: As a tree-based ensemble method, XGBoost effectively captures complex relationships in the data, making it less prone to overfitting compared to simpler models.

# Results, Findings, and Learnings

**Test Data Price Prediction**

In [ ]:
# Finally, we select XGBoost and print the prediction results to another file

# 1. Preparing test data
X_test_df = test_df.copy()  
# Use a copy of test_df, keeping all features

# 2. Use the best model to make predictions
y_pred_log_price = best_model.predict(X_test_df)  
# predict log_price

# 3. change log_price to price
y_pred_price = np.exp(y_pred_log_price)  
# Convert log_price to price using exponential function

# 4. Create a new DataFrame to save the prediction results
results_df = pd.DataFrame({
    'id': test_ID,  
    # Extract id from reserved test_ID
    'predicted_price': y_pred_price  
    # Adding predicted price
})

# 5. Save the prediction results to a CSV file, making sure the id is in the first column
results_df.to_csv('predicted_prices.csv', index=False)  
# Do not keep index

# 6. Print prediction results
print(results_df.head()) 

Important Features for Determining the House Price

1. Grade: [level of construction and design] 
It is  reasonable that a  house in  good construction and design quality will  be more attractive to consumers, resulting in a higher price for sale.

2. Zipcode: [the average price per square meter in the area]
Represent the geographic location, areas close to city centers, lakes, or transportation hubs typically have higher property values.

3. sqft_living: [Square footage of the apartments interior living space] 
The  larger the house, the greater amount of money customers need to pay.